In [ ]:
import tensorflow as tf
tf.__version__


'2.2.0'

In [ ]:
import IPython.display as display
 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
 
import numpy as np
import PIL.Image
import time
import functools
from PIL import Image

In [ ]:
def model1(content_image,style_image):
          
      content_image= Image.open(content_image)
      style_image= Image.open(style_image)
      
      content_image = content_image.convert('RGB')
      style_image = style_image.convert('RGB')


     
      (content_image).save('content.jpg')
      (style_image).save('style.jpg')

      content_path="/content/content.jpg"    #path where the image is stored
      style_path="/content/style.jpg"


 
      def load_img(path_to_img):
        max_dim = 512
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
 
        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim
 
        new_shape = tf.cast(shape * scale, tf.int32)
 
        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img
 
      content_image= load_img(content_path)
      style_image= load_img(style_path)
      
      def tensor_to_image(tensor):
        tensor = tensor*255
        tensor = np.array(tensor, dtype=np.uint8)
        if np.ndim(tensor)>3:
          assert tensor.shape[0] == 1
          tensor = tensor[0]
        return PIL.Image.fromarray(tensor)
 
 
            
 
      def imshow(image, title=None):
        if len(image.shape) > 3:
          image = tf.squeeze(image, axis=0)
 
        plt.imshow(image)
        if title:
          plt.title(title)
 
 
      content_layers = ['block5_conv2'] 
 
      style_layers = ['block1_conv1',
                      'block2_conv1',
                      'block3_conv1', 
                      'block4_conv1', 
                      'block5_conv1']
 
      num_content_layers = len(content_layers)
      num_style_layers = len(style_layers)
 
 
      def vgg_layers(layer_names):
        """ Creates a vgg model that returns a list of intermediate output values."""
        # Load our model. Load pretrained VGG, trained on imagenet data
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False
        
        outputs = [vgg.get_layer(name).output for name in layer_names]
 
        model = tf.keras.Model([vgg.input], outputs)
        return model
 
 
      style_extractor = vgg_layers(style_layers)
      style_outputs = style_extractor(style_image*255)
 
 
      def gram_matrix(input_tensor):
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)
 
 
      class StyleContentModel(tf.keras.models.Model):
        def __init__(self, style_layers, content_layers):
          super(StyleContentModel, self).__init__()
          self.vgg =  vgg_layers(style_layers + content_layers)
          self.style_layers = style_layers
          self.content_layers = content_layers
          self.num_style_layers = len(style_layers)
          self.vgg.trainable = False
 
        def call(self, inputs):
          "Expects float input in [0,1]"
          inputs = inputs*255.0
          preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
          outputs = self.vgg(preprocessed_input)
          style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                            outputs[self.num_style_layers:])
 
          style_outputs = [gram_matrix(style_output)
                          for style_output in style_outputs]
 
          content_dict = {content_name:value 
                          for content_name, value 
                          in zip(self.content_layers, content_outputs)}
 
          style_dict = {style_name:value
                        for style_name, value
                        in zip(self.style_layers, style_outputs)}
          
          return {'content':content_dict, 'style':style_dict}
 
      extractor = StyleContentModel(style_layers, content_layers)
 
      results = extractor(tf.constant(content_image))
 
      style_targets = extractor(style_image)['style']
      content_targets = extractor(content_image)['content']
 
 
      image = tf.Variable(content_image)
 
      def clip_0_1(image):
        return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)
 
      opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
 
      style_weight=1e-2
      content_weight=1e4
 
      def style_content_loss(outputs):
          style_outputs = outputs['style']
          content_outputs = outputs['content']
          style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                                for name in style_outputs.keys()])
          style_loss *= style_weight / num_style_layers
 
          content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                                  for name in content_outputs.keys()])
          content_loss *= content_weight / num_content_layers
          loss = style_loss + content_loss
          return loss
 
      def high_pass_x_y(image):
        x_var = image[:,:,1:,:] - image[:,:,:-1,:]
        y_var = image[:,1:,:,:] - image[:,:-1,:,:]
 
        return x_var, y_var
 
 
      def total_variation_loss(image):
        x_deltas, y_deltas = high_pass_x_y(image)
        return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))
 
      total_variation_loss(image).numpy()
 
      total_variation_weight=30
 
 
      @tf.function()
      def train_step(image):
        with tf.GradientTape() as tape:
          outputs = extractor(image)
          loss = style_content_loss(outputs)
          loss += total_variation_weight*tf.image.total_variation(image)

        grad = tape.gradient(loss, image)
        opt.apply_gradients([(grad, image)])
        image.assign(clip_0_1(image))
        
 
 
      image = tf.Variable(content_image)
 
 
      import time
      start = time.time()
 
      epochs = 10
      steps_per_epoch = 100
 
      step = 0
      for n in range(epochs):
        for m in range(steps_per_epoch):
          step += 1
          train_step(image)
          print(".", end='')
        display.clear_output(wait=True)
        display.display(tensor_to_image(image))
        print("Train step: {}".format(step))
      end = time.time()
 
      display.display(tensor_to_image(image))
 
      file_name = 'stylized-image.png'
      tensor_to_image(image).save(file_name)

In [ ]:
""" Get frontend files from google drive """
 
!mkdir templates
!mkdir static
!cp /content/drive/My\ Drive/StyleTransfer/index.html templates/
!cp /content/drive/My\ Drive/StyleTransfer/generated.html templates/
!cp /content/drive/My\ Drive/StyleTransfer/style.css static/
!cp /content/drive/My\ Drive/StyleTransfer/function.js static/

In [ ]:
""" Get public url """
 
!pip install pyngrok
!ngrok authtoken <yourcodehere>
from pyngrok import ngrok
public_url=ngrok.connect(port="5000")
 
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


'http://f167d623f962.ngrok.io'

In [ ]:
""" Backend api """

from flask import Flask, request, render_template, url_for
from PIL import Image
 
app = Flask(__name__)
 
@app.route('/')
@app.route('/index')
def index():
 
    return render_template('index.html')
 
@app.route('/program', methods=['GET', 'POST'])
def program():
    if request.method == 'POST':
        if request.files:
 
            content = request.files.get('file1','')
            style = request.files.get('file2','')
            model1(content, style)
 
    return render_template('index.html')
 
@app.route('/generated', methods=['GET', 'POST'])
def generated():
    return render_template('generated.html', content='/stylized-image.png')
 
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
